In [1]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.stem.porter import *
from nltk.stem import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chenzhengyi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import json
import os,glob

raw_documents = []
file_order = []
for filename in glob.glob(os.path.join('*.json')):
    #print(filename)
    with open(filename, 'r') as f:
        text = json.load(f)
        file_num = filename.split('.')[0]
        file_order.append(file_num)
        raw_documents.append(text)


In [3]:
print(len(raw_documents))

179


In [4]:
document = []
for doc in raw_documents:
    data_str = ""
    for i in range(len(doc)):
        data_str += doc[i]['transcript']
       
    document.append(data_str)
    

In [5]:
#full_data = []
#full_data.append(data_l)

In [6]:
document[0]

" we're going to Angel Grove not there boy haven't found Krispy Kreme yet. Billy Billy just the two of us"

Write a function to perform lemmatize and stem preprocessing steps on the data set.

In [7]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [8]:
doc_sample = document[0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))
processed_data = preprocess(doc_sample)

original document: 
['', "we're", 'going', 'to', 'Angel', 'Grove', 'not', 'there', 'boy', "haven't", 'found', 'Krispy', 'Kreme', 'yet.', 'Billy', 'Billy', 'just', 'the', 'two', 'of', 'us']


 tokenized and lemmatized document: 
['go', 'angel', 'grove', 'haven', 'krispi', 'kreme', 'billi', 'billi']


## Bag of Words on the Data set

Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set.

In [9]:
#processed_data = data_str.split(" ")
processed_document = []
for doc in document:
    processed_doc = preprocess(doc)
    processed_document.append(processed_doc)

In [10]:
dictionary = gensim.corpora.Dictionary(processed_document)

#count = 0
#for k, v in dictionary.iteritems():
#    print(k, v)
#    count += 1
#    if count > 10:
#        break

## Filter out tokens that appear in

In [11]:
# dictionary.filter_extremes(no_below=1, no_above=0.5, keep_n=100000)

Gensim doc2bow

For each document we create a dictionary reporting how many
words and how many times those words appear. Save this to ‘bow_corpus’, then check our selected document earlier.

In [12]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_document]

In [13]:
bow_corpus[0]
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

Word 0 ("angel") appears 1 time.
Word 1 ("billi") appears 2 time.
Word 2 ("go") appears 1 time.
Word 3 ("grove") appears 1 time.
Word 4 ("haven") appears 1 time.
Word 5 ("kreme") appears 1 time.
Word 6 ("krispi") appears 1 time.


## TF-IDF

Create tf-idf model object using models.TfidfModel on ‘bow_corpus’ and save it to ‘tfidf’, then apply transformation to the entire corpus and call it ‘corpus_tfidf’. Finally we preview TF-IDF scores for our first document.

In [14]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.26175782322444485),
 (1, 0.714446803771632),
 (2, 0.05990451155333907),
 (3, 0.357223401885816),
 (4, 0.1861031420804852),
 (5, 0.357223401885816),
 (6, 0.357223401885816)]


## Running LDA using Bag of Words
Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’

In [15]:
# 10
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [16]:
lda_model = gensim.models.LdaMulticore([bow_corpus[1]], num_topics=5, id2word=dictionary, passes=2, workers=2)

In [17]:
type(bow_corpus)

list

In [18]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.000*"like" + 0.000*"play" + 0.000*"challeng" + 0.000*"game" + 0.000*"alphago" + 0.000*"board" + 0.000*"world" + 0.000*"number" + 0.000*"atom" + 0.000*"pull"
Topic: 1 
Words: 0.000*"board" + 0.000*"challeng" + 0.000*"play" + 0.000*"like" + 0.000*"world" + 0.000*"number" + 0.000*"game" + 0.000*"pull" + 0.000*"alphago" + 0.000*"atom"
Topic: 2 
Words: 0.003*"world" + 0.002*"challeng" + 0.002*"play" + 0.002*"game" + 0.002*"alphago" + 0.002*"board" + 0.002*"number" + 0.002*"like" + 0.001*"simplest" + 0.001*"underway"
Topic: 3 
Words: 0.000*"challeng" + 0.000*"number" + 0.000*"world" + 0.000*"board" + 0.000*"like" + 0.000*"play" + 0.000*"game" + 0.000*"alphago" + 0.000*"atom" + 0.000*"pull"
Topic: 4 
Words: 0.001*"world" + 0.001*"like" + 0.001*"number" + 0.001*"board" + 0.001*"alphago" + 0.000*"game" + 0.000*"play" + 0.000*"challeng" + 0.000*"pull" + 0.000*"long"


## Running LDA using TF-IDF

In [19]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.002*"game" + 0.001*"alphago" + 0.001*"match" + 0.001*"human" + 0.001*"think" + 0.001*"thing" + 0.001*"intellig" + 0.001*"work" + 0.001*"happen" + 0.001*"like"
Topic: 1 Word: 0.001*"song" + 0.001*"bigger" + 0.001*"alphago" + 0.001*"game" + 0.001*"human" + 0.001*"like" + 0.001*"lanka" + 0.001*"match" + 0.001*"move" + 0.001*"go"
Topic: 2 Word: 0.002*"alphago" + 0.001*"game" + 0.001*"play" + 0.001*"learn" + 0.001*"human" + 0.001*"program" + 0.001*"robot" + 0.001*"intellig" + 0.001*"player" + 0.001*"googl"
Topic: 3 Word: 0.001*"know" + 0.001*"think" + 0.001*"game" + 0.001*"insur" + 0.001*"patient" + 0.001*"alphago" + 0.001*"guy" + 0.001*"play" + 0.001*"place" + 0.001*"data"
Topic: 4 Word: 0.002*"alphago" + 0.002*"game" + 0.002*"match" + 0.001*"compani" + 0.001*"surpris" + 0.001*"china" + 0.001*"robot" + 0.001*"googl" + 0.001*"program" + 0.001*"deepmind"


## Performance evaluation by classifying sample document using LDA Bag of Words model
We will check where our test document would be classified.

In [20]:
for index, score in sorted(lda_model[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.983322262763977	 
Topic: 0.003*"world" + 0.002*"challeng" + 0.002*"play" + 0.002*"game" + 0.002*"alphago"


In [21]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 5)))


Score: 0.9829550981521606	 
Topic: 0.002*"alphago" + 0.001*"game" + 0.001*"play" + 0.001*"learn" + 0.001*"human"


In [22]:
lda_model_tfidf.print_topic(4,5)

'0.002*"alphago" + 0.002*"game" + 0.002*"match" + 0.001*"compani" + 0.001*"surpris"'

## Loop!

In [23]:
target_word = ['alphago','googl','game','go']
spam_lst = []

In [24]:
for i in range(len(bow_corpus)):
    lda_model = gensim.models.LdaMulticore([bow_corpus[i]], num_topics=3, id2word=dictionary, passes=2, workers=2)
    topic_lst = []
    for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        #print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))
        topic_lst.append(lda_model.print_topic(index, 5))
    spam = True
    for tgt_word in target_word:
        for topic in topic_lst:
            if tgt_word in topic:
                spam = False
                break
    if spam:
        spam_lst.append(int(file_order[i]))
        

/Users/chenzhengyi/opt/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: invalid value encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [25]:
spam_lst = [int(i) for i in spam_lst]


In [26]:
import pandas as pd
filtered_file = pd.read_excel('/Users/chenzhengyi/Desktop/CU_Video_research/English_vedio_summary.xlsx',header = None, index_col = None, dtype={'file_num':str,'descrip':str, 'descrip2':str})[0] 
wrong_culture_file = pd.read_excel('/Users/chenzhengyi/Desktop/CU_Video_research/English_vedio_summary.xlsx',header = None, sheet_name = 1, index_col = None, dtype={'file_num':str,'descrip':str, 'descrip2':str})[0] 

In [27]:
filtered_file_lst = list(filtered_file)
wrong_culture_file_lst = list(wrong_culture_file)

In [28]:
fail_to_filter = []
success_to_filter = []
wrong_filter = []
for filtered_spam in spam_lst:
    if filtered_spam in filtered_file_lst: #success filter
        success_to_filter.append(filtered_spam)
    else: #filtered but not spam
        wrong_filter.append(filtered_spam)
        
for filtered_f in filtered_file_lst:
    if filtered_f not in spam_lst: # fail to filter
        fail_to_filter.append(filtered_f)

In [29]:
len(success_to_filter)

95

In [30]:
len(wrong_filter)

2

In [31]:
len(fail_to_filter)

67

In [32]:
wrong_filter

[64, 121]

In [33]:
fail_to_filter[1:10]

[5, 9, 10, 13, 14, 19, 20, 22, 26]

5.json

In [34]:
lda_model = gensim.models.LdaMulticore([bow_corpus[165]], num_topics=5, id2word=dictionary, passes=2, workers=2)
topic_lst = []
for index, score in sorted(lda_model[bow_corpus[165]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.9964248538017273	 
Topic: 0.009*"game" + 0.005*"like" + 0.005*"time" + 0.003*"exist" + 0.003*"think"


In [35]:
file_order.index('5')

165

10.json

In [36]:
file_order.index('10')

47

In [37]:
lda_model = gensim.models.LdaMulticore([bow_corpus[47]], num_topics=5, id2word=dictionary, passes=2, workers=2)
topic_lst = []
for index, score in sorted(lda_model[bow_corpus[47]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.9916605949401855	 
Topic: 0.005*"game" + 0.004*"play" + 0.004*"hour" + 0.003*"work" + 0.003*"control"


### Seems that target word 'game' is creating huge problem! remove it from the list

In [38]:
target_word = ['alphago','googl','go','comput','intellig','machin','match'] #similar to whitelist
spam_lst = []
filtered_file = pd.read_excel('/Users/chenzhengyi/Desktop/CU_Video_research/English_vedio_summary.xlsx',header = None, index_col = None, dtype={'file_num':str,'descrip':str, 'descrip2':str})[0] 
wrong_culture_file = pd.read_excel('/Users/chenzhengyi/Desktop/CU_Video_research/English_vedio_summary.xlsx',header = None, sheet_name = 1, index_col = None, dtype={'file_num':str,'descrip':str, 'descrip2':str})[0] 

In [39]:
for i in range(len(bow_corpus)):
    lda_model = gensim.models.LdaMulticore([bow_corpus[i]], num_topics=5, id2word=dictionary, passes=2, workers=2)
    topic_lst = []
    for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        topic_lst.append(lda_model.print_topic(index, 5))
    spam = True
    for tgt_word in target_word:
        for topic in topic_lst:
            if tgt_word in topic:
                spam = False
                break
            
    if spam:
        spam_lst.append(int(file_order[i]))
spam_lst = [int(i) for i in spam_lst]

/Users/chenzhengyi/opt/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: invalid value encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [40]:
fail_to_filter = []
success_to_filter = []
wrong_filter = []
for filtered_spam in spam_lst:
    if filtered_spam in filtered_file_lst: #success filter
        success_to_filter.append(filtered_spam)
    else: #filtered but not spam
        wrong_filter.append(filtered_spam)
        
for filtered_f in filtered_file_lst:
    if filtered_f not in spam_lst: #fail to filter
        fail_to_filter.append(filtered_f)

In [41]:
len(success_to_filter)

98

In [42]:
len(wrong_filter)

0

In [43]:
len(fail_to_filter)

64

In [44]:
len(spam_lst)

98

In [45]:
wrong_filter

[]

It shows that the methods successfully filtered out 98 transcripts(documents) and all of them are in the unrelated list. It means that it achieves precision of 1. 